In [ ]:
import torch

## 3.1.1 텐서의 차원 자유자재로 다루기

In [3]:
x = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Size: torch.Size([3, 3])
Shape: torch.Size([3, 3])
랭크(차원): 2


In [4]:
x = torch.unsqueeze(x, 0)
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])
Size: torch.Size([1, 3, 3])
Shape: torch.Size([1, 3, 3])
랭크(차원): 3


In [5]:
x = torch.squeeze(x, 0)
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Size: torch.Size([3, 3])
Shape: torch.Size([3, 3])
랭크(차원): 2


In [6]:
x = x.view(9)
print(x)
print("Size:", x.size())
print("Shape:", x.shape)
print("랭크(차원):", x.ndimension())

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
Size: torch.Size([9])
Shape: torch.Size([9])
랭크(차원): 1


In [7]:
try:
    x = x.view(2,4)
except Exception as e:
    print(e) #에러출력

shape '[2, 4]' is invalid for input of size 9


## 3.1.2  텐서를 이용한 연산과 행렬곱

In [8]:
w = torch.randn(5,3, dtype=torch.float)
x = torch.tensor([[1.0,2.0],[3.0,4.0],[5.0,6.0]])
print("w size:",w.size())
print("x size:",x.size())
print("w:",w)
print("x:",x)

w size: torch.Size([5, 3])
x size: torch.Size([3, 2])
w: tensor([[-2.0490, -0.6096, -0.5557],
        [-1.2732,  0.5708, -1.8098],
        [ 0.3502, -0.6999,  1.3430],
        [ 1.7297, -0.3794, -0.7756],
        [-1.6721, -1.1553,  0.6792]])
x: tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


w는 정규분포(normal distribution)에서 무작위로 값을 뽑아 텐서를 생성하는 randn() 함수에 5와 3을 인수로 전달하여 5x3의 shape를 가진 텐서를 만든다.

무작위로 값을 뽑을 때 실수의 범위 내에서 뽑도록 randn() 함수의 인수 dtype에 torch.float를 지정한다.

In [9]:
b = torch.randn(5, 2, dtype=torch.float)
print('b size:', b.size())
print("b:", b)

b size: torch.Size([5, 2])
b: tensor([[-1.4885,  2.0395],
        [-1.2022, -1.5805],
        [-0.0304,  0.9215],
        [-0.1375, -0.3068],
        [-0.1078,  1.1404]])


* 행렬곱은 torch.mm() 함수를 사용해 수행한다.

In [10]:
wx = torch.mm(w,x) # w의 행은 5, x의 열은 2, 즉 shape는 [5, 2]입니다.
print("wx size:", wx.size())
print("wx:", wx)

wx size: torch.Size([5, 2])
wx: tensor([[ -6.6566,  -9.8710],
        [ -8.6099, -11.1221],
        [  4.9653,   5.9586],
        [ -3.2864,  -2.7117],
        [ -1.7419,  -3.8900]])


In [11]:
result = wx + b
print("result size:", result.size())
print("result:", result)

result size: torch.Size([5, 2])
result: tensor([[ -8.1452,  -7.8316],
        [ -9.8121, -12.7026],
        [  4.9349,   6.8801],
        [ -3.4240,  -3.0185],
        [ -1.8497,  -2.7496]])


## 3.1.3 Autograd



In [12]:
w = torch.tensor(1.0, requires_grad = True)

requires_grad를 True 값으로 설정하면 파이토치의 Autograd 기능이 자동으로 계산할 때 w에 대한 미분값을 w.grad에 저장한다.

In [14]:
a = w*3

In [16]:
l = a**2

In [17]:
l.backward()
print('l을 w로 미분한 값은 {}'.format(w.grad))

l을 w로 미분한 값은 18.0


backward() 메서드는 연쇄법칙을 통한 미분을 지원한다.

# 3.2 경사하강법으로 이미지 복원하기

## 3.2.1 오염된 이미지 문제

이미지 처리를 위해 만들어 두었던 weird_function() 함수에 실수로 버그가 들어가 100x100 픽셀의 오염된 이미지가 만들어졌습니다.

이 오염된 이미지와 오염되기 전 원본 이미지를 동시에 파일로 저장하려고 했으나, 모종의 이유로 원본 이미지 파일은 삭제된 상황입니다.

다행히도 weird_function() 함수의 소스코드는 남아 있습니다.

오염된 이미지와 weird_function() 함수를 활용해서 원본 이미지를 복원해보자.

## 3.2.2 오염된 이미지를 복원하는 방법